In [1]:
import torch
from torch import nn
from transformers import AutoTokenizer, AutoModelForCausalLM

/home/aj3051/anaconda3/envs/backpack/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/aj3051/anaconda3/envs/backpack/lib/python3.11/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [2]:
device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")

In [3]:
tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/deepseek-coder-1.3b-base", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("deepseek-ai/deepseek-coder-1.3b-base", trust_remote_code=True).to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
module_dict = dict(model.named_modules())
second_to_last_attn_layer = module_dict['model.layers.22']

In [ ]:
t0, t1 = None, None

In [13]:
def print_output_dimensions_hook(module, input, output):
    print(len(output))
    global t0, t1
    t0 = output[0]
    t1 = output[1]
output_hook_handle = second_to_last_attn_layer.register_forward_hook(print_output_dimensions_hook)

In [12]:
output_hook_handle.remove()

In [14]:
input = torch.randint(100, size=(4, 189)).to(model.device)

In [15]:
output = model(input)

2
tensor([[[ 6.1546e-01, -5.7934e-01,  5.3252e-01,  ...,  7.5423e-01,
           4.0738e-01, -7.0639e-02],
         [ 3.3488e-01, -3.1896e+00,  1.6580e+00,  ..., -6.9320e+00,
           1.3070e+00,  2.7097e-01],
         [-4.1879e+00, -5.0339e+00, -1.2603e+00,  ..., -3.7969e+00,
           4.5087e+00, -3.6279e+00],
         ...,
         [-5.2951e+00, -3.4597e+00, -1.6850e+00,  ..., -4.0988e+00,
           1.7761e+00, -3.3192e+00],
         [-4.3696e+00, -1.6827e+00, -3.4729e+00,  ..., -2.1099e+00,
           3.2392e-01,  8.4261e-01],
         [-3.2664e+00, -2.6713e+00, -4.0747e+00,  ..., -5.3469e+00,
           1.2561e+00, -1.8406e-01]],

        [[-4.1013e-03, -5.4911e-01,  4.8409e-01,  ...,  1.2295e+00,
           1.9161e-01,  2.7434e-01],
         [ 2.6457e+00, -3.8138e+00, -2.8669e-01,  ...,  6.1548e-01,
          -1.5887e+00,  6.3306e-01],
         [ 4.7496e-01, -2.9190e+00, -4.4240e+00,  ...,  8.8504e-02,
          -1.0563e+00, -1.6418e+00],
         ...,
         [-3.5923e+00, 

In [31]:
output_tensor = output[0]
output_tensor.shape

torch.Size([4, 189, 32256])

In [19]:
embedding = model.model.embed_tokens(input)
pre_permute = embedding


In [27]:
test = torch.arange(800).reshape(4, 10, 20)

In [21]:
x = model.model.layers[0](position_ids=pre_permute)

TypeError: LlamaDecoderLayer.forward() missing 1 required positional argument: 'hidden_states'

In [15]:
pre_permute = embedding
for i in range(23):
    pre_permute = model.model.layers[i](pre_permute)

AttributeError: 'NoneType' object has no attribute 'float'

In [34]:
model.model.embed_tokens.weight

Parameter containing:
tensor([[ 1.8188e-02,  2.0898e-01, -5.9326e-02,  ..., -9.1797e-02,
          1.4648e-01,  1.4160e-01],
        [-3.1006e-02,  1.1133e-01, -7.4219e-02,  ..., -4.7607e-03,
          1.8799e-02,  7.7637e-02],
        [ 1.1230e-01,  2.4609e-01,  6.1768e-02,  ..., -1.3086e-01,
          4.7852e-02,  6.4453e-02],
        ...,
        [-2.3499e-03, -6.0120e-03, -6.4373e-05,  ..., -8.0566e-03,
         -7.3242e-03, -7.4768e-03],
        [ 2.8687e-03,  7.5684e-03,  1.3733e-02,  ...,  1.3924e-04,
         -1.5106e-03, -1.0681e-04],
        [ 8.6670e-03,  1.8997e-03,  5.4169e-04,  ..., -2.4109e-03,
         -6.6833e-03, -3.3264e-03]], device='cuda:2', requires_grad=True)

In [13]:
model.model.layers[:-1]

ModuleList(
  (0-22): 23 x LlamaDecoderLayer(
    (self_attn): LlamaSdpaAttention(
      (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
      (k_proj): Linear(in_features=2048, out_features=2048, bias=False)
      (v_proj): Linear(in_features=2048, out_features=2048, bias=False)
      (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
      (rotary_emb): LlamaLinearScalingRotaryEmbedding()
    )
    (mlp): LlamaMLP(
      (gate_proj): Linear(in_features=2048, out_features=5504, bias=False)
      (up_proj): Linear(in_features=2048, out_features=5504, bias=False)
      (down_proj): Linear(in_features=5504, out_features=2048, bias=False)
      (act_fn): SiLU()
    )
    (input_layernorm): LlamaRMSNorm()
    (post_attention_layernorm): LlamaRMSNorm()
  )
)

In [25]:
model.model.norm.weights

AttributeError: 'LlamaRMSNorm' object has no attribute 'weights'

In [13]:
module_dict = dict(model.named_modules())
type(module_dict['model.layers.22'])

transformers.models.llama.modeling_llama.LlamaDecoderLayer

In [42]:
model.model.norm.weight

Parameter containing:
tensor([0.3477, 0.3574, 0.3711,  ..., 0.3691, 0.3750, 0.3672], device='cuda:2',
       requires_grad=True)

In [15]:
for name, module in model.named_modules():
    print(f'NAME: {name} MODULE: {module}')
    

NAME:  MODULE: LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32256, 2048)
    (layers): ModuleList(
      (0-23): 24 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (v_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaLinearScalingRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=5504, bias=False)
          (up_proj): Linear(in_features=2048, out_features=5504, bias=False)
          (down_proj): Linear(in_features=5504, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaR

In [63]:
for name, module in model.named_modules():
    # Check if the module is the one you want to add a hook to
    # print(name)
    if name == 'model.layers.22':
        handle = module.register_forward_hook(print_dims)
    

In [ ]:
module.remove_forward_hook()

In [66]:
def create_hook(name):
    def print_submodule_name_hook(module, input, output):
        print(f"{module.__class__.__name__} ({name})")
    return print_submodule_name_hook

for name, submodule in model.named_modules():
    submodule.register_forward_hook(create_hook(name))

In [68]:
inputs = tokenizer(input_text, return_tensors="pt")['input_ids'].to(model.device)


In [69]:
outputs = model(inputs, output_hidden_states=True)

Embedding (lm_head)
Embedding (model.embed_tokens)
Embedding (model.embed_tokens)
LlamaRMSNorm (lm_head)
LlamaRMSNorm (model.layers.0.input_layernorm)
LlamaRMSNorm (model.layers.0.input_layernorm)
Linear (lm_head)
Linear (model.layers.0.self_attn.q_proj)
Linear (model.layers.0.self_attn.q_proj)
Linear (lm_head)
Linear (model.layers.0.self_attn.k_proj)
Linear (model.layers.0.self_attn.k_proj)
Linear (lm_head)
Linear (model.layers.0.self_attn.v_proj)
Linear (model.layers.0.self_attn.v_proj)
LlamaLinearScalingRotaryEmbedding (lm_head)
LlamaLinearScalingRotaryEmbedding (model.layers.0.self_attn.rotary_emb)
LlamaLinearScalingRotaryEmbedding (model.layers.0.self_attn.rotary_emb)
Linear (lm_head)
Linear (model.layers.0.self_attn.o_proj)
Linear (model.layers.0.self_attn.o_proj)
LlamaSdpaAttention (lm_head)
LlamaSdpaAttention (model.layers.0.self_attn)
LlamaSdpaAttention (model.layers.0.self_attn)
LlamaRMSNorm (lm_head)
LlamaRMSNorm (model.layers.0.post_attention_layernorm)
LlamaRMSNorm (model.